In [1]:
import pandas as pd
import numpy as np
import mlflow 
from pathlib import Path
from typing import Tuple

In [2]:
# load processed train and dev data for modelling
def load_data(filepath: str, dataset_type: str) -> pd.DataFrame:
    '''
        Loads processed data from csv source
    
        This function loads processed data from csv file
        that will be used for modelling
        
        Args:
            filepath: path to processed dataset
            dataset_type: whether the dataset is train, dev or test dataframe
            
        Returns:
            pd.DataFrame: returns a dataframe containing processed data
            
        Examples:
            >>> df = load_data('data/processed/train_set.csv')
                df.head()
    '''
    filename = Path(filepath)
    if not filename.exists():
        raise FileNotFoundError(f'File not found! Check filepath and try again later!')
    
    df = pd.read_csv(filename)

    # check that the df is not empty
    if len(df) == 0:
        raise ValueError(f'Dataframe cannot be empty!')
    
    print(f'{dataset_type} dataframe successfully loaded with {df.shape[0]} rows and {df.shape[1]} features')
    
    return df


TRAIN_DATA_PATH = '../data/processed/train_set.csv'
DEV_DATA_PATH = '../data/processed/dev_set.csv'

dev_df = load_data(DEV_DATA_PATH, 'Dev')
train_df = load_data(TRAIN_DATA_PATH, 'Train')



Dev dataframe successfully loaded with 7500 rows and 44 features
Train dataframe successfully loaded with 35000 rows and 44 features


In [ ]:
# perform one last data quality check before modelling
def data_quality_checks(df: pd.DataFrame) -> None:
    '''
        Performs one last data quality check before modelling

        Args:
            df: pandas' dataframe to be validated
        
        Raises:
            ValueError: if any of the validations fail

        Example:
            >>> data_quality_checks(df)
    '''

    missing = df.isnull().sum().sum()
    if missing:
        raise ValueError("The dataset must not contain null values")
    
    n_duplicates = df.duplicated().sum()
    if n_duplicates != 0:
        raise ValueError("The dataset must not contain duplicate rows")

    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if len(numerical_cols) != len(df.columns):
        raise ValueError("All columns in the dataset must be numerical")

    print("All validation passed!")

data_quality_checks(train_df)
data_quality_checks(dev_df)

All validation passed!
All validation passed!


In [4]:
# split the data further into features and target set
def features_target_split(df: pd.DataFrame, target: str = 'Current_Salary') -> Tuple[pd.DataFrame, pd.DataFrame]:
    '''
        Split the dataset further into features and target splits
        
        This function takes the given dataframe and splits it
        into the feature set and target set for modelling purposes
    
        Args:
            df: pd.DataFrame = Provided dataframe
            target: str = Target variable in the given dataframe

        Returns:
            A tuple of the features and target sets

        Examples:
            >>> x, y = features_target_split(df, 'Employee_salary')
                x.head()
                y.head()
    '''
    y = df[target].copy()
    if not isinstance(y, pd.Series):
        raise ValueError('The target variable must be a pandas series')
    
    if len(y) == 0:
        raise ValueError(f'The target variable cannot be empty')
    
    x = df.drop(columns=[target]).copy()
    if not isinstance(x, pd.DataFrame):
        raise ValueError('The feature set must be a pandas dataframe')
    
    if len(x) == 0:
        raise ValueError(f'The target variable cannot be empty')
    
    if len(y) != len(x):
        raise ValueError('The length of the target variable must be equal to the length of the feature set')

    
    return x, y

x_train, y_train = features_target_split(train_df)
x_dev, y_dev = features_target_split(dev_df)

In [5]:
print('x_train: ', x_train.shape)
print(f'x_dev: {x_dev.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_dev: {y_dev.shape}')

x_train:  (35000, 43)
x_dev: (7500, 43)
y_train: (35000,)
y_dev: (7500,)


In [6]:
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, learning_curve
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error,mean_absolute_error,root_mean_squared_error,r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import time

In [7]:
# baseline models - performing modelling with minimal feature engineering
models = {
    'Ridge' : Ridge(random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=200, max_depth=6, n_jobs=-1, random_state=1),
    'XGBoost': XGBRegressor(),
    'LightGBM': LGBMRegressor(random_state=2)
}


In [9]:

for model_name, model in models.items():
    print(f'Training model: {model_name}...')
    start_time = time.time()
    model.fit(x_train, y_train)

    time_elapsed = time.time() - start_time

    preds_train = model.predict(x_train)
    # preds_dev = model.predict(x_dev)

    results = {
        'Model_name' : model_name,
        'Mean_squared_error' : mean_squared_error(y_train, preds_train),
        'Root_mean_squared_error' : root_mean_squared_error(y_train, preds_train),
        'Mean_absolute_error' : mean_absolute_error(y_train, preds_train),
        'Time_elapsed' : time_elapsed
    }
    
    for name, result in results.items():
        print(f'{name}_train : {result}')
        
    print('='*50)

Training model: Ridge...
Model_name_train : Ridge
Mean_squared_error_train : 177006236.93275234
Root_mean_squared_error_train : 13304.369091871751
Mean_absolute_error_train : 5950.650044137561
Time_elapsed_train : 0.04302263259887695
Training model: RandomForest...
Model_name_train : RandomForest
Mean_squared_error_train : 374677.1392871908
Root_mean_squared_error_train : 612.1087642626846
Mean_absolute_error_train : 299.84564094744934
Time_elapsed_train : 39.75776767730713
Training model: XGBoost...
Model_name_train : XGBoost
Mean_squared_error_train : 31568.296875
Root_mean_squared_error_train : 177.67469787597656
Mean_absolute_error_train : 100.116455078125
Time_elapsed_train : 1.7549805641174316
Training model: LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005956 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not 

In [10]:
# cross validation score 
cv = KFold(n_splits=5, shuffle=True, random_state=9999)

for name, model in models.items():
    cross_val = cross_val_score(model, x_train, y_train, cv=cv, scoring='neg_root_mean_squared_error', n_jobs=-1)

    cv_score = -cross_val
    results = {
        'model_name' : name,
        'cv_score_across_fold' : cv_score,
        'cv_score_mean' : cv_score.mean(),
        'cv_score_std' : cv_score.std()
    }

    for name, result in results.items():
        print(f'{name}_train : {result}')

    print('='*50)

model_name_train : Ridge
cv_score_across_fold_train : [13214.01516076 11546.08185647 14546.91316514 12515.08319543
 14545.629717  ]
cv_score_mean_train : 13273.544618960095
cv_score_std_train : 1166.4135659553087
model_name_train : RandomForest
cv_score_across_fold_train : [631.03277737 819.79787692 757.55650098 574.76559407 682.24520977]
cv_score_mean_train : 693.0795918224054
cv_score_std_train : 87.38442501704962
model_name_train : XGBoost
cv_score_across_fold_train : [9402.90039062 6235.70605469 8811.60839844 9500.25976562 9101.83203125]
cv_score_mean_train : 8610.461328125
cv_score_std_train : 1211.6987123585957
model_name_train : LightGBM
cv_score_across_fold_train : [7905.2099425  6079.50476953 7564.06289146 5803.99762713 7063.48003909]
cv_score_mean_train : 6883.251053943197
cv_score_std_train : 818.6798231414747
